In [1]:
import tensorflow as tf
import time
import tensorflow_hub as hub
import cv2
from PIL import Image
import uuid

from EfficientDet import *
import os
from glob import glob

# 특정 GPU만 사용

In [2]:
# gpus = tf.config.experimental.list_physical_devices("GPU")
# if gpus:
# 	# 텐서플로가 세번째 GPU만 사용하도록 제한
# 		try:
# 				tf.config.experimental.set_visible_devices(gpus[-1], "GPU")
# 		except :
# 				print(e)

# pretrained model 가져오기

In [3]:
model = get_detector()

In [4]:
dataset_path = "./original_train/"
output_path = "./detected_data/detected_from_train/"
dataset_list = os.listdir("./original_train")

# object detection > 탐지된 객체 저장

In [5]:
detected_train_formats, draw_imgs = object_detection(model, dataset_list, dataset_path, output_path)

person: 0.8226
Detection 수행시간: 5.42 초
clock: 0.6318
Detection 수행시간: 5.43 초
person: 0.4391
Detection 수행시간: 5.44 초
laptop: 0.9128
Detection 수행시간: 0.09 초
person: 0.6140
Detection 수행시간: 0.09 초
bed: 0.5120
Detection 수행시간: 0.09 초
teddy bear: 0.8622
Detection 수행시간: 0.07 초
person: 0.7681
Detection 수행시간: 0.08 초
bed: 0.6660
Detection 수행시간: 0.09 초
vase: 0.4740
Detection 수행시간: 0.09 초
person: 0.8905
Detection 수행시간: 0.07 초
person: 0.8225
Detection 수행시간: 0.08 초
bench: 0.4663
Detection 수행시간: 0.08 초
person: 0.4076
Detection 수행시간: 0.09 초
giraffe: 0.9180
Detection 수행시간: 0.08 초
person: 0.4186
Detection 수행시간: 0.08 초
chair: 0.5963
Detection 수행시간: 0.08 초
umbrella: 0.5629
Detection 수행시간: 0.08 초
chair: 0.4851
Detection 수행시간: 0.09 초
umbrella: 0.4699
Detection 수행시간: 0.09 초
chair: 0.4614
Detection 수행시간: 0.1 초
person: 0.4431
Detection 수행시간: 0.1 초
umbrella: 0.4196
Detection 수행시간: 0.11 초
chair: 0.4153
Detection 수행시간: 0.11 초
umbrella: 0.4151
Detection 수행시간: 0.12 초
person: 0.8109
Detection 수행시간: 0.08 초
person: 0.8093


In [7]:
import pickle


In [8]:
with open("detected_train_formats.p", "wb") as out:
    pickle.dump(detected_train_formats, out)
    

In [9]:

with open("./detected_train_formats.p","rb") as f:
    detected_train_formats = pickle.load(f)

from glob import glob
image_path = [format["objectImagePath"] for format in detected_train_formats]
detected_images_train = [Image.open(path) for path in image_path]

In [10]:
from ImageRetrievalClass import ImageRetrievalClass

retrievalInstance = ImageRetrievalClass("EfficientNet", True, False)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [11]:
retrievalInstance.readTrainSet(detected_images_train)

Reading train images
train image shape = (256, 256, 3)


In [12]:
retrievalInstance.buildModel()

Loading EfficientNet pre-trained model...
Model: "efficientnetb7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 256, 256, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization[0][0]              
___________________________________________

In [13]:
X_train = retrievalInstance.trainTransform()

Applying image transformer to training images...
 -> X_train.shape = (4850, 256, 256, 3)


In [14]:
np.save("./X_train.npy",X_train)

In [15]:
E_train = retrievalInstance.predictTrain(X_train) # 연결이 끊기네... (256, 256)으로 바꾸니 해결됨

Inferencing embeddings using pre-trained model...
 -> E_train.shape = (4850, 8, 8, 2560)


# 객체 각각의 npy 파일 저장

In [18]:
for idx, format in enumerate(detected_train_formats):
    filename = format["objectImagePath"][36:-4]
    npyFilename = "./npyFiles/"+filename+".npy"
    np.save(npyFilename, E_train[idx])


# JSON 파일 저장

In [19]:
import json
for idx, format in enumerate(detected_train_formats):
    npyFilename = "./npyFiles/"+format["objectImagePath"][36:-4]+".npy"
    jsonDict = {
        "objectID":format["objectID"],
        "location":format["location"],
        "tag":format["tag"],
        "objectImagePath" : format["objectImagePath"],
        "npyPath": npyFilename,
        "IMG_URL" : format["IMG_URL"]
    }

    filename = "./jsonFiles/"+format["objectImagePath"][36:-4]+".json"

    with open(filename, "w") as outFile:
        json.dump(jsonDict, outFile, indent=2)